# American Sign Language (ASL) Image Recognition

## Introduction

## Loading in dataset

In [1]:
import re
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFile  
from sklearn.datasets import load_files
from keras.utils import np_utils
from keras.preprocessing import image                  
from tqdm import tqdm


models_dir = 'saved_models'
data_dir = 'data'

Using TensorFlow backend.


### Dividing data into training, validation, and testing sets

Now that preprocessing the images is completed (see `data_preprocessing.ipynb` notebook), the full dataset will be split into training, validation, and testing sets. The testing set will be all the images from one subject to mirror the "Spelling It Out" paper's method so the benchmark model can be compared. The rest of the images will be randomly split; 80% of images for training, 20% of the images for validation.

In [2]:
def load_dataset(path, n_categories=24):
    data = load_files(path)
    image_files = np.array(data['filenames'])
    # Hot encode categories to matrix
    image_targets = np_utils.to_categorical(np.array(data['target']), n_categories)
    return image_files, image_targets

def move_data_by_category(container_dir, regex_file_format='.*png'):
    '''Move data into a directory based on category'''
    # Still check if files are images
    file_list = [x for x in os.listdir(container_dir) if re.search(regex_file_format, x)]
    # Get numerical string (note that 1 digits are represented w/ 2 digits) 
    letters = {x.split('_')[0] for x in file_list}
    
    for letter in letters:
        # Only images that match letter
        images_with_letter = [filename for filename in file_list if filename.split('_')[0] == letter]
        # Add images to sub directory
        new_categ_path = os.path.join(container_dir, letter)
        if not os.path.exists(new_categ_path):
            os.makedirs(new_categ_path)
        print(f'Created {new_categ_path} dir with {len(images_with_letter)} items')
        for img_filename in images_with_letter:
            path = os.path.join(container_dir, img_filename)
            new_path = os.path.join(new_categ_path, img_filename)            
            os.rename(path, new_path)
    # TODO: Check if any files were skipped (improperly named?)
        

def get_testing_data(data_dir, subject_num='4'):
    '''Get all data/images pertaining to one subject'''
    # Only search in directory for images with that subject
    file_list = [x for x in os.listdir(data_dir) if re.search(f'\d+_{subject_num}_\d*\.png', x)]
    
    # Make a new testing data directory if doesn't exist
    testing_dir = os.path.join(data_dir, 'test')
    if not os.path.exists(testing_dir):
        os.makedirs(testing_dir)
        
    # Move images of particular subject into testing directory
    for image_filename in file_list:
        # file is **_n_****.png where n is an integer representing a subject
        _, subject, _ = image_filename.split('_')
        # Move file into testing directory
        path = os.path.join(data_dir, image_filename)
        new_path = os.path.join(testing_dir, image_filename)
        os.rename(path, new_path)
        
    # Move each image file's numerical str representing letters found in testing into own category directory
    move_data_by_category(testing_dir)
    
    return load_dataset(testing_dir)


def get_training_validation_data(data_dir, ratio=0.8):
    '''Randomly split data into training and validation sets'''
    # Only search in directory for images
    file_list = [x for x in os.listdir(data_dir) if re.search('.*png', x)]
    
    # Make a new training & validation data directory if doesn't exist
    train_dir = os.path.join(data_dir, 'train')
    valid_dir = os.path.join(data_dir, 'valid')
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(valid_dir):
        os.makedirs(valid_dir)
        
    # Randomly split file list into training and vaidation sets
    np.random.shuffle(file_list)
    split_int = int(ratio * len(file_list))
    train_list = file_list[:split_int]
    valid_list = file_list[split_int:]
    
    # Move images of particular subject into testing directory
    for filenames, new_dir in [(train_list, train_dir), (valid_list, valid_dir)]:
        for image_filename in filenames:
            # Move file into testing directory
            path = os.path.join(data_dir, image_filename)
            new_path = os.path.join(new_dir, image_filename)
            os.rename(path, new_path)

        # Move each image file's numerical str representing letters found in testing into own category directory
        move_data_by_category(new_dir)
    
    return (load_dataset(train_dir), load_dataset(valid_dir))

In [3]:
test_data, test_targets = get_testing_data(data_dir)
train, valid = get_training_validation_data(data_dir)
# Separated data and its targets
train_data, train_targets = train
valid_data, valid_targets = valid

### Display some of the images

In [ ]:
np.random.seed(8675309)
%matplotlib inline

# Display image previews below
plt.figure(figsize=(20,55))
columns = 8
n = 1

# Randomly choose images to display (with label)
for image_path in np.random.choice(train_data, 24, replace=False):
    img = Image.open(image_path)
    plt.subplot(20, columns, n)
    n+=1
    plt.imshow(img)
    letter = image_path.split('/')[-1][:2]
    letter = chr(int(letter)+65)
    plt.title(letter)

### Preparing the model

In [35]:
# Preprocessing data fro Keras (TensorFlow backend)
def path_to_tensor(img_path):
    # Loads image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224), grayscale=False)
    # Convert PIL.Image.Image type to 3D tensor with shape (224, 224, 1)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 1) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [36]:
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
valid_tensors = paths_to_tensor(valid_data).astype('float32')
test_tensors = paths_to_tensor(test_data).astype('float32')

100%|██████████| 13898/13898 [00:13<00:00, 995.85it/s] 


In [37]:
# Prepare training data separately since this is usually large
train_tensors = paths_to_tensor(train_data).astype('float32')

100%|██████████| 41500/41500 [01:05<00:00, 637.43it/s]


## Basic CNN model from scratch

### Building basic model

In [38]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()
#Convo 224, 224, 1
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(224, 224, 1)))
#
model.add(MaxPooling2D(pool_size=2))
#
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
#
model.add(MaxPooling2D(pool_size=2))
#
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
#
model.add(MaxPooling2D(pool_size=2))
#
model.add(GlobalAveragePooling2D())
#Dense; 24 for each handshape 
model.add(Dense(24, activation='softmax'))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 160, 160, 16)      80        
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 80, 80, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 80, 80, 32)        2080      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 40, 40, 64)        8256      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
global_average_pooling2d_3 ( (None, 64)                0         
__________

In [39]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### Training basic model

In [40]:
from keras.callbacks import ModelCheckpoint  

epochs = 16

# Create a saved models directory
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

checkpointer = ModelCheckpoint(filepath=f'{models_dir}/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 41500 samples, validate on 10376 samples
Epoch 1/16
41500/41500 [==============================] - 72s 2ms/step - loss: 2.7341 - acc: 0.1881 - val_loss: 2.3031 - val_acc: 0.2963
Epoch 2/16
41500/41500 [==============================] - 71s 2ms/step - loss: 2.1256 - acc: 0.3397 - val_loss: 1.9848 - val_acc: 0.3566
Epoch 3/16
41500/41500 [==============================] - 71s 2ms/step - loss: 1.8238 - acc: 0.4251 - val_loss: 1.9093 - val_acc: 0.3714
Epoch 4/16
41500/41500 [==============================] - 71s 2ms/step - loss: 1.5803 - acc: 0.4980 - val_loss: 1.4175 - val_acc: 0.5447
Epoch 5/16
41500/41500 [==============================] - 71s 2ms/step - loss: 1.3728 - acc: 0.5605 - val_loss: 1.1916 - val_acc: 0.6340
Epoch 6/16
41500/41500 [==============================] - 71s 2ms/step - loss: 1.2068 - acc: 0.6133 - val_loss: 1.0598 - val_acc: 0.6657
Epoch 7/16
41500/41500 [==============================] - 71s 2ms/step - loss: 1.0752 - acc: 0.6520 - val_loss: 1.0159 - val_acc

### Evaluating basic model

In [41]:
handshape_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(handshape_predictions)==np.argmax(test_targets, axis=1))/len(handshape_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 43.3156%


## Transfer Learning with VGG16

### Preparing the model

In [ ]:
# Preprocessing data fro Keras (TensorFlow backend)
def path_to_tensor(img_path):
    # Loads image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(80, 80), grayscale=False)
    # Convert PIL.Image.Image type to 3D tensor with shape (80, 80, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 80, 80, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True                 
# Pre-process the data for Keras
# Prepare training data separately since this is usually large

print('Preparing to create valid tensors')
valid_tensors = paths_to_tensor(valid_data).astype('float32')
print('valid tensors prepared')

In [ ]:
print('Preparing to create test tensors')
test_tensors = paths_to_tensor(test_data).astype('float32')
print('test tensors prepared')

In [ ]:
print('Preparing to create train tensors')
train_tensors = paths_to_tensor(train_data).astype('float32')
print('train tensors prepared')

### Extract Bottleneck Features for Training Set

In [29]:
# Load the data
targets = np.squeeze(train_targets)
print('data loaded')

# Load vgg16 model + remove final classification layers
model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(80, 80, 3), classes=24)
print('model loaded')

# Obtain bottleneck features (train)
if os.path.exists('vgg16_features_train.npz'):
    print('bottleneck features detected (train)')
    features = np.load('vgg16_features_train.npz')['features']
else:
    print('bottleneck features file not detected (train)')
    
    print('calculating now ...')
    # Pre-process the train data
    big_x_train = np.array([scipy.misc.imresize(train_tensors[i], (80, 80, 3)) 
                            for i in range(0, len(train_tensors))]).astype('float32')
    vgg16_input_train = vgg16.preprocess_input(big_x_train)
    print('train data preprocessed')
    
    # Extract, process, and save bottleneck features
    features = model.predict(vgg16_input_train)
    features = np.squeeze(features)
    np.savez('vgg16_features_train', features=features)

print('bottleneck features saved (train)')

data loaded
model loaded
bottleneck features file not detected (train)
calculating now ...
train data preprocessed
bottleneck features saved (train)


### Extract Bottleneck Features for Training Set

In [30]:
# Obtain bottleneck features (test)
if os.path.exists('inception_features_test.npz'):
    print('bottleneck features detected (test)')
    features_test = np.load('inception_features_test.npz')['features_test']
else:
    # Calculating for validation set
    print('bottleneck features file not detected (valid)')
    print('calculating now ...')
    # pre-process the test data
    big_x_valid = np.array([scipy.misc.imresize(valid_tensors[i], (80, 80, 3)) 
                       for i in range(0, len(valid_tensors))]).astype('float32')
    # big_x_valid = valid_tensors

    inception_input_valid = preprocess_input(big_x_valid)
    # extract, process, and save bottleneck features (valid)
    features_valid = model.predict(inception_input_valid)
    features_valid = np.squeeze(features_valid)
    np.savez('inception_features_valid', features_test=features_valid)
print('bottleneck features saved (valid)')

bottleneck features file not detected (valid)
calculating now ...
bottleneck features saved (valid)


### Building model

In [32]:
# Shallow NN
from keras.callbacks import ModelCheckpoint   
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, GlobalAveragePooling2D

model = Sequential()
model.add(Conv2D(filters=100, kernel_size=2, input_shape=features.shape[1:]))
model.add(Dropout(0.4))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(24, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 1, 1, 100)         204900    
_________________________________________________________________
dropout_9 (Dropout)          (None, 1, 1, 100)         0         
_________________________________________________________________
global_average_pooling2d_5 ( (None, 100)               0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 24)                2424      
Total params: 207,324
Trainable params: 207,324
Non-trainable params: 0
_________________________________________________________________


In [18]:
checkpointer = ModelCheckpoint(filepath='model.best_batch500_epoch150.hdf5', 
                               verbose=1, save_best_only=True)
          
model.fit(features, targets, batch_size=500, epochs=150,
          validation_data=(features_valid, valid_targets),
          callbacks=[checkpointer],
          verbose=2, shuffle=True)          

Train on 41500 samples, validate on 10376 samples
Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.23283, saving model to model.best_batch500_epoch150.hdf5
 - 1s - loss: 0.2613 - acc: 0.9629 - val_loss: 0.2328 - val_acc: 0.9692
Epoch 2/150
Epoch 00002: val_loss did not improve
 - 1s - loss: 0.2332 - acc: 0.9654 - val_loss: 0.2516 - val_acc: 0.9652
Epoch 3/150
Epoch 00003: val_loss did not improve
 - 1s - loss: 0.2314 - acc: 0.9654 - val_loss: 0.2381 - val_acc: 0.9688
Epoch 4/150
Epoch 00004: val_loss did not improve
 - 1s - loss: 0.2299 - acc: 0.9659 - val_loss: 0.2532 - val_acc: 0.9677
Epoch 5/150
Epoch 00005: val_loss improved from 0.23283 to 0.22631, saving model to model.best_batch500_epoch150.hdf5
 - 1s - loss: 0.2198 - acc: 0.9672 - val_loss: 0.2263 - val_acc: 0.9696
Epoch 6/150
Epoch 00006: val_loss did not improve
 - 1s - loss: 0.2160 - acc: 0.9672 - val_loss: 0.2330 - val_acc: 0.9682
Epoch 7/150
Epoch 00007: val_loss improved from 0.22631 to 0.22434, saving model to mo

Epoch 63/150
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.1189 - acc: 0.9786 - val_loss: 0.2194 - val_acc: 0.9697
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.1192 - acc: 0.9778 - val_loss: 0.2420 - val_acc: 0.9682
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.1240 - acc: 0.9782 - val_loss: 0.2243 - val_acc: 0.9698
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.1147 - acc: 0.9789 - val_loss: 0.2252 - val_acc: 0.9709
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.1183 - acc: 0.9774 - val_loss: 0.2382 - val_acc: 0.9697
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.1171 - acc: 0.9792 - val_loss: 0.2145 - val_acc: 0.9712
Epoch 69/150
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.1183 - acc: 0.9781 - val_loss: 0.2155 - val_acc: 0.9701
Epoch 70/150
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.1157 - acc: 0.9783 - val_loss: 0.2172 - val_acc: 0.9727
Epoch 71/150
Epo

Epoch 129/150
Epoch 00129: val_loss did not improve
 - 1s - loss: 0.0907 - acc: 0.9819 - val_loss: 0.2209 - val_acc: 0.9717
Epoch 130/150
Epoch 00130: val_loss did not improve
 - 1s - loss: 0.0915 - acc: 0.9824 - val_loss: 0.2342 - val_acc: 0.9710
Epoch 131/150
Epoch 00131: val_loss did not improve
 - 1s - loss: 0.0892 - acc: 0.9830 - val_loss: 0.2361 - val_acc: 0.9698
Epoch 132/150
Epoch 00132: val_loss did not improve
 - 1s - loss: 0.0897 - acc: 0.9832 - val_loss: 0.2153 - val_acc: 0.9716
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 1s - loss: 0.0916 - acc: 0.9824 - val_loss: 0.2359 - val_acc: 0.9693
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 1s - loss: 0.0961 - acc: 0.9824 - val_loss: 0.2193 - val_acc: 0.9710
Epoch 135/150
Epoch 00135: val_loss did not improve
 - 1s - loss: 0.0914 - acc: 0.9836 - val_loss: 0.2386 - val_acc: 0.9698
Epoch 136/150
Epoch 00136: val_loss did not improve
 - 1s - loss: 0.0890 - acc: 0.9827 - val_loss: 0.2087 - val_acc: 0.9718
Epoch 13

### Evaluating the model

In [17]:
# Load the weights that yielded the best validation accuracy
model.load_weights('model.best_batch500_epoch150.hdf5')

# Evaluate test accuracy
score = model.evaluate(features_test, test_targets, verbose=0)
accuracy = 100*score[1]

# Print test accuracy
print('Test accuracy: %.4f%%' % accuracy)

Test accuracy: 53.9646%
